In [ ]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

In [ ]:
def preprocess_data(df):
    # Fill NaN values with empty string to avoid issues with TF-IDF
    df = df.fillna("")
    return df

In [ ]:
# Load bird dataset
bird_df = pd.read_csv("birds.csv")
bird_df = preprocess_data(bird_df)

bird_df.head()

In [ ]:
# combine relevant features
bird_df["combined_features"]=bird_df["city"]+" "+bird_df["forest"]+" "+bird_df["Season"]+" "+bird_df["active"]

vectorizer=TfidfVectorizer()
tfidf_matrix=vectorizer.fit_transform(bird_df["combined_features"].values.astype("U"))

In [ ]:
# Save model and vectorizer to pickle file

with open("birds_model.pkl", "wb") as model_file:
    pickle.dump((bird_df, vectorizer, tfidf_matrix), model_file)

In [ ]:
def get_top_birds(location, season, visit_time, top_n=5):
    # Load model from pickle file
    with open("birds_model.pkl", "rb") as model_file:
        wildlife_df, vectorizer, tfidf_matrix = pickle.load(model_file)

    # User query formatted similarly
    user_query = location + " " + season + " " + visit_time
    user_tfidf = vectorizer.transform([user_query])

    # Compute cosine similarity
    similarity_scores = cosine_similarity(user_tfidf, tfidf_matrix).flatten()

    # Get indices of top N matches
    top_indices = similarity_scores.argsort()[-top_n:][::-1]

    # Get the top matching animals along with relevant details
   # Get the top matching birds along with relevant details
    result_df = bird_df.iloc[top_indices][[
    "name","Scientific Name", "Season", "active", "forest", "city", "Region",
    "Unique Fact", "Endemic Status", "Conservation Status"
    ]]

    # Compute accuracy as the average similarity score of the top matches
    accuracy = similarity_scores[top_indices].mean() * 100  # Convert to percentage

    # Convert to HTML table format with custom styling
    table_html = result_df.to_html(index=False, escape=False, classes="styled-table")

    # Append accuracy information
    table_html += f"<p><strong>Accuracy of top {top_n} matches: {accuracy:.2f}%</strong></p>"

    return table_html


In [ ]:
# Prompt user for input
user_location = input("Enter the city/state you are visiting : ")
user_season = input("Enter the season of your visit (summer,winter,mansoon) : ")
user_visit_time = input("Enter the time of day you are visiting (morning, afternoon, night) : ")

In [ ]:
# Get recommendations
top_animals_html = get_top_birds(user_location, user_season, user_visit_time)

In [ ]:
# Print HTML with custom styles
html_style = """
<style>
.styled-table {
    width: 100%;
    border-collapse: collapse;
    font-family: Arial, sans-serif;
}
.styled-table th {
    background-color: #4CAF50;
    color: white;
    padding: 10px;
    text-align: left;
    font-size: 16px;
}
.styled-table td {
    padding: 8px;
    border-bottom: 1px solid #ddd;
}
.styled-table tr:nth-child(even) {
    background-color: #f2f2f2;
}
.styled-table tr:nth-child(odd) {
    background-color: #ffffff;
}
</style>
"""
print(html_style + top_animals_html)